In [251]:
import pandas as pd
import numpy as np
#import seaborn as sns
import math
import os
import datetime
import re
#from google.colab import drive
import seaborn as sns
import matplotlib.pyplot as plt
#drive.mount('/content/drive')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
import tqdm
import scipy.stats as stats

In [112]:
freqs = pd.read_csv('./data/frequencies.csv')

In [161]:
numberRe = re.compile('[0-9]+')
noneRe = re.compile('None')
def daysStrToInt(dStr):
    if isinstance(dStr, str):
        if numberRe.match(dStr):
            return int(dStr.split(' ')[0])
        elif noneRe.match(dStr):
            return None
    return dStr

freqs['hospDistance'] = freqs['hospDistance'].transform(daysStrToInt)

In [162]:
x = freqs.drop(columns=['hasHospitilization', 'hospDistance'])
y = freqs['hasHospitilization']
d = freqs['hospDistance']

In [132]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25, random_state=15)

In [266]:
#model = SVC(kernel='poly', C=10, class_weight='balanced')
#model = SVC(kernel='poly', C=1, class_weight='balanced')
#model = SVC(kernel='linear', C=10, class_weight='balanced')
#model = SVC(kernel='linear', C=1, class_weight='balanced')
#model = SVC(kernel='linear')
#model = DecisionTreeClassifier(criterion='entropy')
#model = KNeighborsClassifier(n_neighbors=5)
model = BaggingClassifier(base_estimator=SVC(kernel='linear'), n_estimators=10, random_state=15)

model.fit(x_train, y_train)

BaggingClassifier(base_estimator=SVC(kernel='linear'), random_state=15)

In [254]:
pred = model.predict(x_test)
#pred = model.predict(x)

In [ ]:
#print(classification_report(y_test, pred))
conf = confusion_matrix(y_true=y_test, y_pred=pred)
plot_confusion_matrix(model, x_test, y_test)
plot_confusion_matrix(model, x_test, y_test, normalize='true')

In [265]:
precision = conf[1,1]/sum(conf[:,1])
recall = conf[1,1]/sum(conf[1])
f1Score = 2/(1/precision + 1/recall)

#Looking for high recall
print(f'Precision: {precision}\nRecall: {recall}\nF1 Score: {f1Score}')

Precision: 0.7692307692307693
Recall: 0.5357142857142857
F1 Score: 0.6315789473684211


In [201]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     13926
         1.0       0.83      0.70      0.76        56

    accuracy                           1.00     13982
   macro avg       0.91      0.85      0.88     13982
weighted avg       1.00      1.00      1.00     13982



In [258]:
def computeDistance(x_vals, model):
    p = model.decision_function(x_vals)
    #w_norm = np.linalg.norm(model.coef_)
    #dist = p / w_norm
    dist = p
    return pd.Series(data=dist)

In [259]:
#dist = computeDistance(x_train, model)
dist = computeDistance(x, model)
graphData = []
step = 20
boundary = min(dist)
rng = max(dist)-boundary
for i in range(0,1+step):
    v = boundary+(i/step)*rng
    # nope # eq included so that 0 and 1 are caught
    #print(f'v: {v}, p: {predVal}')
    sub = dist.loc[(dist >= v) & (dist < v+rng/step)]
    a = {'Prob': v, 'Density': len(sub)}
    #print(graphData)
    graphData.append(a)
graphData = pd.DataFrame(data=graphData)
#graphData = graphData.astype(float)

In [120]:
percentile = stats.norm(dist.mean(), dist.std())
g = pd.DataFrame()
g.insert(0, 'x', dist)
g.insert(1, 'y', percentile.cdf(dist))

In [141]:
g = pd.DataFrame()
g.insert(0, 'x', sorted(dist))
pct = []
for i, v in g['x'].items():
    pct.append(i/len(g['x']))
g.insert(1, 'y', pct)

In [ ]:
g = pd.DataFrame()
g.insert(0, 'x', dist)
#g.insert(1, 'y', d.iloc[x_train.index])
g.insert(1, 'y', d.iloc[x.index])

In [ ]:
plt.figure(figsize=(10,5))
#sns.lineplot(x='x',y='y',data=g)
plt.xlim(-1, 1)
sns.scatterplot(x='x',y='y',data=g,alpha=0.5)

In [103]:
print(f'{(len(g["x"].loc[g["x"] > 0])/len(g["x"]))*100}% positives')

0.348091457454164% positives


In [147]:
dist.index

RangeIndex(start=0, stop=41943, step=1)

In [168]:
g['y'].sort_values().unique()

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15